In [9]:
library(tidyverse)
library(DBI)
library(purrr)
library(jsonlite)

In [20]:
## connect to database
con1 <- DBI::dbConnect(
    RMySQL::MySQL(), 
    host = "localhost",
    user = "uog2000",
    port = 3307,
    dbname = "user_uog2000",
    password = read_file('/home/uog2000/mimir_setup/password.txt') %>% str_replace('\n', '')
)
con <- DBI::dbConnect(
    RMySQL::MySQL(), 
    host = "localhost",
    user = "uog2000",
    port = 3307,
    dbname = "clinical_merge_v5_2024q1",
    password = read_file('/home/uog2000/mimir_setup/password.txt') %>% str_replace('\n', '')
)

In [225]:
referece_ades <- con1 %>% tbl('reference_ades') %>% as_tibble()

In [39]:
reference_rxnorm <- con1 %>% tbl('reference_ades_rxnorm') %>% as_tibble()

In [40]:
reference_rxnorm <- reference_rxnorm %>%
                        mutate(RXCUI = as.integer(RXCUI))
reference_rxnorm %>% filter(is.na(RXCUI))

drug_concept_id,drug_name,concept_id_2,RXCUI,concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept,concept_code,valid_start_date,valid_end_date,invalid_reason
<int>,<chr>,<int>,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>


In [64]:
path <- "../drug_labels/gold_xml/"
files <- dir(path, pattern = "*.json")
gold <- files %>%
       map_df(~fromJSON(file.path(path, .), flatten = TRUE))

In [65]:
path <- "../drug_labels/train_xml/"
files <- dir(path, pattern = "*.json")
train <- files %>%
       map_df(~fromJSON(file.path(path, .), flatten = TRUE))

In [131]:
path <- "../drug_labels/latest_labels_ingredients/"
files <- dir(path, pattern = "*.json")

In [132]:
data <- files %>%
       map_df(~fromJSON(file.path(path, .), flatten = TRUE))
data %>% select(-sections) %>% head()

set_id,label_id,spl_version,title
<chr>,<chr>,<chr>,<chr>
1612d931-d86e-4dba-bbd7-0aac3eaee131,B08BDAA0-28F1-6D59-3526-C965D2DB1A6F,1,HMS®(medrysone ophthalmic suspension) 1%
1612d931-d86e-4dba-bbd7-0aac3eaee131,B08BDAA0-28F1-6D59-3526-C965D2DB1A6F,1,HMS®(medrysone ophthalmic suspension) 1%
1612d931-d86e-4dba-bbd7-0aac3eaee131,B08BDAA0-28F1-6D59-3526-C965D2DB1A6F,1,HMS®(medrysone ophthalmic suspension) 1%
1612d931-d86e-4dba-bbd7-0aac3eaee131,B08BDAA0-28F1-6D59-3526-C965D2DB1A6F,1,HMS®(medrysone ophthalmic suspension) 1%
312042cf-df60-4dc5-90dd-4e77ebd4a781,312042cf-df60-4dc5-90dd-4e77ebd4a781,1,"ENLON (edrophonium chloride injection, USP)Rx only"
312042cf-df60-4dc5-90dd-4e77ebd4a781,312042cf-df60-4dc5-90dd-4e77ebd4a781,1,"ENLON (edrophonium chloride injection, USP)Rx only"


In [182]:
data %>% distinct(set_id) %>% dim()
rbind(data, train, gold) %>% distinct(set_id) %>% dim()

[1] 1551    1

[1] 1634    1

In [183]:
data <- rbind(data, train, gold)

In [184]:
data %>% filter(set_id == '1b5646a9-4724-44b8-8cd0-9c14f5e87f86') %>% select(-sections)

set_id,label_id,spl_version,title
<chr>,<chr>,<chr>,<chr>
1b5646a9-4724-44b8-8cd0-9c14f5e87f86,120fbd67-f644-aa50-e063-6394a90aae42,12,"These highlights do not include all the information needed to use LAMOTRIGINE TABLETS safely and effectively. See full prescribing information for LAMOTRIGINE TABLETS. LAMOTRIGINE Tablets, for oral use Initial U.S. Approval: 1994"
1b5646a9-4724-44b8-8cd0-9c14f5e87f86,120fbd67-f644-aa50-e063-6394a90aae42,12,"These highlights do not include all the information needed to use LAMOTRIGINE TABLETS safely and effectively. See full prescribing information for LAMOTRIGINE TABLETS. LAMOTRIGINE Tablets, for oral use Initial U.S. Approval: 1994"
1b5646a9-4724-44b8-8cd0-9c14f5e87f86,120fbd67-f644-aa50-e063-6394a90aae42,12,"These highlights do not include all the information needed to use LAMOTRIGINE TABLETS safely and effectively. See full prescribing information for LAMOTRIGINE TABLETS. LAMOTRIGINE Tablets, for oral use Initial U.S. Approval: 1994"
1b5646a9-4724-44b8-8cd0-9c14f5e87f86,120fbd67-f644-aa50-e063-6394a90aae42,12,"These highlights do not include all the information needed to use LAMOTRIGINE TABLETS safely and effectively. See full prescribing information for LAMOTRIGINE TABLETS. LAMOTRIGINE Tablets, for oral use Initial U.S. Approval: 1994"
1b5646a9-4724-44b8-8cd0-9c14f5e87f86,120fbd67-f644-aa50-e063-6394a90aae42,12,"These highlights do not include all the information needed to use LAMOTRIGINE TABLETS safely and effectively. See full prescribing information for LAMOTRIGINE TABLETS. LAMOTRIGINE Tablets, for oral use Initial U.S. Approval: 1994"


In [168]:
rxnorm_mappings <- fread('../drug_labels/rxnorm_mappings_20240312.csv.gz')
rxnorm_mappings %>% head()

SETID,SPL_VERSION,RXCUI,RXSTRING,RXTTY
<chr>,<int>,<int>,<chr>,<chr>
000155a8-709c-44e5-a75f-cd890f3a7caf,27,198014,naproxen 500 MG Oral Tablet,PSN
000155a8-709c-44e5-a75f-cd890f3a7caf,27,198014,naproxen 500 MG Oral Tablet,SCD
0001d1cf-f727-4875-a44f-5a51f7b617a4,2,1046593,benzalkonium chloride 0.13 % Medicated Liquid Soap,PSN
0001d1cf-f727-4875-a44f-5a51f7b617a4,2,1046593,benzalkonium chloride 0.13 % Medicated Liquid Soap,SY
0001d1cf-f727-4875-a44f-5a51f7b617a4,2,1046593,benzalkonium chloride 1.3 MG/ML Medicated Liquid Soap,SCD
0001d1cf-f727-4875-a44f-5a51f7b617a4,2,1046593,benzalkonium chloride 1.33 MG/ML Medicated Liquid Soap,SY


In [190]:
rxnorm_mappings %>% filter(SETID == '1b5646a9-4724-44b8-8cd0-9c14f5e87f86')

SETID,SPL_VERSION,RXCUI,RXSTRING,RXTTY
<chr>,<int>,<int>,<chr>,<chr>
1b5646a9-4724-44b8-8cd0-9c14f5e87f86,12,282401,lamoTRIgine 25 MG Oral Tablet,PSN
1b5646a9-4724-44b8-8cd0-9c14f5e87f86,12,282401,lamotrigine 25 MG Oral Tablet,SCD


In [191]:
mapped_drugs <- rxnorm_mappings %>%
    inner_join(reference_rxnorm, by = 'RXCUI') %>%
    distinct(drug_concept_id, drug_name, RXCUI)

Warning message in inner_join(., reference_rxnorm, by = "RXCUI"):
“Each row in `x` is expected to match at most 1 row in `y`.
ℹ Row 734 of `x` matches multiple rows.
ℹ If multiple matches are expected, set `multiple = "all"` to silence this
  warning.”


In [192]:
labeled_drugs <- rxnorm_mappings %>%
    inner_join(data, c('SETID' = 'set_id')) %>%
    inner_join(reference_rxnorm, by = 'RXCUI') %>%
    distinct(drug_concept_id, drug_name, RXCUI)

Warning message in inner_join(., data, c(SETID = "set_id")):
“Each row in `x` is expected to match at most 1 row in `y`.
ℹ Row 24 of `x` matches multiple rows.
ℹ If multiple matches are expected, set `multiple = "all"` to silence this
  warning.”


In [193]:
rxnorm_mappings %>%
    inner_join(train, by = c('SETID' = 'set_id'), multiple = 'all') %>%
    inner_join(reference_rxnorm, by = 'RXCUI') %>%
    distinct(drug_concept_id, drug_name) %>% dim()

[1] 3 2

In [245]:
mapped_drugs %>% distinct(drug_concept_id, drug_name) %>% dim()
labeled_drugs %>% distinct(drug_concept_id, drug_name) %>% dim()
referece_ades %>% distinct(drug_concept_id, drug_name) %>% dim()

[1] 173   2

[1] 159   2

[1] 182   2

In [195]:
reference_ades %>% as_tibble() %>% mutate(RXCUI = as.integer(RXCUI)) %>%
    left_join(mapped_drugs %>% mutate(mapped = 1)) %>%
    distinct(drug_concept_id, drug_name, mapped) %>% 
    left_join(labeled_drugs %>% distinct(drug_concept_id, drug_name) %>%
              mutate(label = 1), by = c('drug_concept_id', 'drug_name')) %>%
distinct(drug_concept_id, drug_name, mapped, label) %>%
    replace_na(list('mapped' = 0, 'label' = 0)) %>%
    group_by(drug_concept_id, drug_name) %>%
    summarise(seen = 1,
              mapped = max(mapped, na.rm = T),
              label = max(label, na.rm = T)) %>%
filter(mapped == 0, label == 0)

Joining with `by = join_by(drug_concept_id, drug_name, RXCUI)`
`summarise()` has grouped output by 'drug_concept_id'. You can override using the
`.groups` argument.


drug_concept_id,drug_name,seen,mapped,label
<int>,<chr>,<dbl>,<dbl>,<dbl>
727835,Pemoline,1,0,0
953391,Propantheline,1,0,0
1103374,valdecoxib,1,0,0
1721543,Norfloxacin,1,0,0
1724827,Zalcitabine,1,0,0
19008009,Cosyntropin,1,0,0
19011093,Tenofovir,1,0,0
19026710,Capreomycin,1,0,0


In [198]:
data %>% filter(set_id == '1b5646a9-4724-44b8-8cd0-9c14f5e87f86') %>% select(-sections)

set_id,label_id,spl_version,title
<chr>,<chr>,<chr>,<chr>
1b5646a9-4724-44b8-8cd0-9c14f5e87f86,120fbd67-f644-aa50-e063-6394a90aae42,12,"These highlights do not include all the information needed to use LAMOTRIGINE TABLETS safely and effectively. See full prescribing information for LAMOTRIGINE TABLETS. LAMOTRIGINE Tablets, for oral use Initial U.S. Approval: 1994"
1b5646a9-4724-44b8-8cd0-9c14f5e87f86,120fbd67-f644-aa50-e063-6394a90aae42,12,"These highlights do not include all the information needed to use LAMOTRIGINE TABLETS safely and effectively. See full prescribing information for LAMOTRIGINE TABLETS. LAMOTRIGINE Tablets, for oral use Initial U.S. Approval: 1994"
1b5646a9-4724-44b8-8cd0-9c14f5e87f86,120fbd67-f644-aa50-e063-6394a90aae42,12,"These highlights do not include all the information needed to use LAMOTRIGINE TABLETS safely and effectively. See full prescribing information for LAMOTRIGINE TABLETS. LAMOTRIGINE Tablets, for oral use Initial U.S. Approval: 1994"
1b5646a9-4724-44b8-8cd0-9c14f5e87f86,120fbd67-f644-aa50-e063-6394a90aae42,12,"These highlights do not include all the information needed to use LAMOTRIGINE TABLETS safely and effectively. See full prescribing information for LAMOTRIGINE TABLETS. LAMOTRIGINE Tablets, for oral use Initial U.S. Approval: 1994"
1b5646a9-4724-44b8-8cd0-9c14f5e87f86,120fbd67-f644-aa50-e063-6394a90aae42,12,"These highlights do not include all the information needed to use LAMOTRIGINE TABLETS safely and effectively. See full prescribing information for LAMOTRIGINE TABLETS. LAMOTRIGINE Tablets, for oral use Initial U.S. Approval: 1994"


In [199]:
rxnorm_mappings %>%
    inner_join(data %>% select(-sections), by = c('SETID' = 'set_id')) %>%
    distinct(SETID) %>% dim()

Warning message in inner_join(., data %>% select(-sections), by = c(SETID = "set_id")):
“Each row in `x` is expected to match at most 1 row in `y`.
ℹ Row 24 of `x` matches multiple rows.
ℹ If multiple matches are expected, set `multiple = "all"` to silence this
  warning.”


[1] 1604    1

In [214]:
reference_labels %>% ungroup() %>% distinct(drug_concept_id) %>% dim()

[1] 159   1

In [207]:
reference_labels <- rxnorm_mappings %>%
    inner_join(reference_rxnorm, by = 'RXCUI', multiple = "all") %>%
    inner_join(data, by = c('SETID'='set_id'), multiple = "all") %>%
    inner_join(referece_ades, by = c('drug_concept_id', 'drug_name'), multiple = "all") %>% 
    group_by(cohort_id, condition_name, drug_concept_id, drug_name, affect,
             set_id=SETID, label_id, spl_version, title) %>%
    summarise(sections = paste0(sections, collapse = '\n')) %>%
    select('cohort_id','condition_name','drug_concept_id','drug_name',
           'affect','set_id','label_id','spl_version','title','sections') %>%
# write_csv('reference_set_labels.csv')

`summarise()` has grouped output by 'cohort_id', 'condition_name',
'drug_concept_id', 'drug_name', 'affect', 'set_id', 'label_id', 'spl_version'. You
can override using the `.groups` argument.


In [217]:
referece_ades %>% distinct(condition_name, drug_concept_id) %>%
    left_join(reference_labels %>% ungroup() %>% mutate(retrieved = 1)) %>%
    replace_na(list('retrieved' = 0)) %>%
    distinct(condition_name, drug_concept_id, retrieved) %>%
    group_by(condition_name) %>%
    summarise(n_drugs = n_distinct(drug_concept_id),
              n_retrieved = sum(retrieved),
              prop = round(n_retrieved/n_drugs * 100, digits = 2))

Joining with `by = join_by(condition_name, drug_concept_id)`
Warning message in left_join(., reference_labels %>% ungroup() %>% mutate(retrieved = 1)):
“Each row in `x` is expected to match at most 1 row in `y`.
ℹ Row 17 of `x` matches multiple rows.
ℹ If multiple matches are expected, set `multiple = "all"` to silence this
  warning.”


condition_name,n_drugs,n_retrieved,prop
<chr>,<int>,<dbl>,<dbl>
Acute kidney injury,88,76,86.36
Acute liver injury,118,101,85.59
Acute myocardial infarction,102,89,87.25
GI bleed,91,78,85.71


In [219]:
reference_labels %>% ungroup() %>% distinct(drug_concept_id) %>% dim()

[1] 159   1

In [221]:
referece_ades %>% ungroup() %>% distinct(drug_concept_id) %>% dim()

[1] 182   1

In [227]:
rxnorm_mappings %>% head()

SETID,SPL_VERSION,RXCUI,RXSTRING,RXTTY
<chr>,<int>,<int>,<chr>,<chr>
000155a8-709c-44e5-a75f-cd890f3a7caf,27,198014,naproxen 500 MG Oral Tablet,PSN
000155a8-709c-44e5-a75f-cd890f3a7caf,27,198014,naproxen 500 MG Oral Tablet,SCD
0001d1cf-f727-4875-a44f-5a51f7b617a4,2,1046593,benzalkonium chloride 0.13 % Medicated Liquid Soap,PSN
0001d1cf-f727-4875-a44f-5a51f7b617a4,2,1046593,benzalkonium chloride 0.13 % Medicated Liquid Soap,SY
0001d1cf-f727-4875-a44f-5a51f7b617a4,2,1046593,benzalkonium chloride 1.3 MG/ML Medicated Liquid Soap,SCD
0001d1cf-f727-4875-a44f-5a51f7b617a4,2,1046593,benzalkonium chloride 1.33 MG/ML Medicated Liquid Soap,SY


In [233]:
referece_ades %>% 
    distinct(drug_concept_id, drug_name) %>%
    anti_join(reference_labels) %>%
write_csv('missing_labels.csv')

Joining with `by = join_by(drug_concept_id, drug_name)`


In [238]:
temp <- referece_ades %>% 
    distinct(drug_concept_id, drug_name) %>%
    anti_join(reference_labels)
missing <- paste0(unique(temp$drug_name), collapse = '|')

Joining with `by = join_by(drug_concept_id, drug_name)`


In [240]:
data %>% rowwise() %>%
    mutate(title = title %>% tolower()) %>%
    filter(grepl(pattern = missing, x = title)) %>%
    mutate(drug_name = str_match(title, missing)[1]) %>% ungroup()

set_id,label_id,spl_version,sections,title,drug_name
<chr>,<chr>,<chr>,<named list>,<chr>,<chr>


In [243]:
referece_ades %>% as_tibble() %>%
    anti_join(reference_ades %>% as_tibble())

Joining with `by = join_by(drug_concept_id, drug_name)`


cohort_id,condition_name,drug_concept_id,drug_name,affect
<int>,<chr>,<int>,<chr>,<int>
500000401,Acute kidney injury,1312742,Olmesartan medoxomil,1
